In [123]:
from sklearn import tree # módulo de arvore de decisao
from sklearn import preprocessing # Modulo de preprocessamento
from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection
import pandas as pd # Pandas manipulação de dados
import os # acesso ao sistema operacional
import graphviz
import numpy as np

In [47]:
df = pd.read_csv( 'dia05/dados_candidatos.csv', sep=';', decimal=',' )

target = 'descricao_grau_instrucao'
features = [ 'descricao_cargo', 'descricao_cor_raca', 'descricao_estado_civil', 'descricao_genero',
                'sigla_uf_nascimento', 'nome_partido', 'patrimonio'  ]

group_names = { 'LE E ESCREVE': 'SEM ESCOLARIDADE',
                'ENSINO FUNDAMENTAL INCOMPLETO': 'SEM ESCOLARIDADE',
                'ENSINO MEDIO INCOMPLETO':'ENSINO FUNDAMENTAL COMPLETO',
                'SUPERIOR INCOMPLETO': 'ENSINO MEDIO COMPLETO' }

df[target] = df[target].apply( lambda x: group_names[x] if x in group_names else x )

# Agrupa suplentes
df['descricao_cargo'] = df['descricao_cargo'].replace( ['1o SUPLENTE SENADOR', '2o SUPLENTE SENADOR'], 'SUPLENTE SENADOR' )

In [55]:
cat_features = df[features].dtypes[df[features].dtypes == 'object'].index
onehot = preprocessing.OneHotEncoder(sparse=False)
onehot.fit( df[cat_features ] )
X = pd.DataFrame( onehot.transform( df[cat_features] ), columns = onehot.get_feature_names( cat_features ) )

num_features = df[features].dtypes[df[features].dtypes != 'object'].index
X[num_features] = df[num_features].copy()
X['declara_patrimonio'] = X['patrimonio'] > 0

In [88]:
df['sample1'] = np.random.random( df.shape[0] ) < 0.6
clf_1 = tree.DecisionTreeClassifier(max_depth=10, min_samples_leaf=20)
clf_1.fit( X[ df['sample1'] ] , df[target][df['sample1']] )

df['sample2'] = np.random.random( df.shape[0] ) < 0.6
clf_2 = tree.DecisionTreeClassifier(max_depth=10, min_samples_leaf=20)
clf_2.fit( X[ df['sample2'] ] , df[target][df['sample2']] )

predict_full = clf.predict( X.iloc[0:3] )
predict_1 = clf_1.predict( X.iloc[0:3] )
predict_2 = clf_2.predict( X.iloc[0:3] )

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=20, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

# Definindo base de treino e base de teste

In [125]:
X_train, X_test, y_train, y_test = model_selection.train_test_split( X, df[target], test_size=0.2)

In [135]:
X_test.shape[0] + X_train.shape[0] == X.shape[0]

True

In [132]:
clf = tree.DecisionTreeClassifier()
clf.fit( X_train, y_train ) 

clf_rf = ensemble.RandomForestClassifier( n_estimators=500, max_depth=20, n_jobs=3 )
clf_rf.fit( X_train, y_train )

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=20, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=3,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [133]:
print(" Acurácia para random forest treino:", metrics.accuracy_score(  y_train , clf_rf.predict(X_train) ) )
print(" Acurácia para Arvore de decisao treino:", metrics.accuracy_score(  y_train , clf.predict(X_train) ) )

Acurácia para random forest treino: 0.7284682826199071
 Acurácia para Arvore de decisao treino: 0.926766374419804


In [134]:
print(" Acurácia para random forest treino:", metrics.accuracy_score(  y_test, clf_rf.predict(X_test) ) )
print(" Acurácia para Arvore de decisao treino:", metrics.accuracy_score(  y_test , clf.predict(X_test) ) )

Acurácia para random forest treino: 0.568678012721334
 Acurácia para Arvore de decisao treino: 0.4626095925734915
